# Comparison of performance Online Algorithms

**Simulations of randomly generated group sequences on a selection of grids**

In [1]:
from utils.test_file import generate_group_sequences
from algorithms.online import FirstFit, BestFit, WorstFit, Greedy, Hybrid
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
import itertools
from online_batch import run_algorithm_with_original_groups, repeat_algorithm_with_different_groups, get_file_names

## Set variables For simulation

We need to define:

* which files (=grids) we are going to run
* how many random groups we gonna test per grids
* the number of groups
* which algorithms

In [2]:
FILE_DIR = "input/online"
RANDOM_SEED = "Ultimate Ushers"
NUMBER_OF_GROUP_SEQUENCES = 200
LENGTH_OF_SEQUENCE = 200
# 20 different group sequences from each 50 groups long
GROUPS_LIST = generate_group_sequences(NUMBER_OF_GROUP_SEQUENCES, LENGTH_OF_SEQUENCE, RANDOM_SEED)
ALGORITHMS = [
    FirstFit,
    BestFit,
    WorstFit,
    Greedy,
    Hybrid
]

## Run the simulation

Gather all the files that we want to run on.

In [3]:
file_names = get_file_names(FILE_DIR)
grid_order = ['Online1', 'Online2', 'Online3', 'Online4', 'Online5', 'Online6', 'Online7', 'Online8', 'Online9', 'Online10', 'Online11', 'Online12', 'Online13', 'Online14', 'Online15']
# print(file_names)

Run the algorithms and gather the results in a dataframe

In [4]:
%%capture
result = {}
for file in file_names:
    grid_path = f"{FILE_DIR}/{file}"

    grid_result = {}
    for algorithm in ALGORITHMS:
        alg_filled_chairs = []
        alg_filled_chairs.append(run_algorithm_with_original_groups(algorithm, grid_path))
        alg_filled_chairs = alg_filled_chairs + repeat_algorithm_with_different_groups(algorithm, grid_path, GROUPS_LIST)
        alg_name = str(algorithm.__name__)
        grid_result.update({alg_name: alg_filled_chairs})
    
    result.update({file: grid_result})


In [5]:
print("The following grids were solved:")    
print(result.keys())

The following grids were solved:
dict_keys(['Online1.txt', 'Online10.txt', 'Online11.txt', 'Online12.txt', 'Online13.txt', 'Online14.txt', 'Online15.txt', 'Online2.txt', 'Online3.txt', 'Online4.txt', 'Online5.txt', 'Online6.txt', 'Online7.txt', 'Online8.txt', 'Online9.txt'])


Put everything in a dataframe:

In [6]:
df_list = []
for grid, algs in result.items():
    alg_series = []
    for alg, value_list in algs.items():
        alg_series.append(pd.Series(value_list, name=alg))
    df = pd.DataFrame(alg_series).transpose()
    df = df.assign(grid=grid[0:-4]).set_index('grid', append=True, drop=True)
    df_list.append(df)
df = pd.concat(df_list)

## Preprocessing

Let's extract the records with the original groups from the input grid files, and put it in a seperate dataframe

In [4]:
# The first group is the original group included in the grid file
df_real_groups = pd.read_pickle("results/online_vs_offline.p")
# df_real_groups = df.xs(0, level=None)
df_real_groups.loc[grid_order].to_latex("results/table_online_grids.tex")

And export the dataframe, so it can be used elsewhere to do comparison with results from offline algorithm

In [8]:
df_real_groups.to_pickle("results/online_vs_offline.p")


Gather the other data to separate dataframe for the generated group simulation records: 

In [9]:
# All the groups except group [0] belongs to the simulation
df_sim = df.loc[pd.IndexSlice[range(1,len(GROUPS_LIST)+1),:]]
df_sim.to_pickle("results/online_sim_results.p")

In [5]:
df_sim = pd.read_pickle("results/online_sim_results.p")
df_sim

FirstFit  BestFit  WorstFit  Greedy  Hybrid
    grid                                                 
1   Online1          4        5         3       7       7
    Online10        72       65        67      76      67
    Online11        58       64        51      63      64
    Online12       132      133       124     143     138
    Online13       235      242       183     247     242
...                ...      ...       ...     ...     ...
200 Online5         27       25        24      25      25
    Online6         29       31        26      32      31
    Online7         41       42        32      39      42
    Online8         45       46        46      48      47
    Online9         67       64        62      66      61

[3000 rows x 5 columns]

## Analysis

**The following analyses are on the simulation results.**

### Mean seats filled per algorithm per grid

The mean is taken over all the group sequences that were generated

In [10]:
df_sim_mean = df_sim.groupby(['grid']).mean().reset_index().melt(id_vars='grid', value_name='seats', var_name='algorithm')
# df_sim_avg

In [11]:
df_sim_std = df_sim.groupby(['grid']).std().reset_index().melt(id_vars='grid', value_name='std', var_name='algorithm')
# df_sim_std

In [15]:
df_sim_comb = df_sim_mean.merge(df_sim_std, on=['grid', 'algorithm']).set_index('grid')
df_sim_comb = df_sim_comb.loc[grid_order].reset_index()
df_sim_comb


,grid,algorithm,seats,std
0,Online1,FirstFit,5.120,0.985115
1,Online1,BestFit,5.575,1.039122
2,Online1,WorstFit,4.335,0.993838
3,Online1,Greedy,6.600,0.575898
4,Online1,Hybrid,6.600,0.575898
...,...,...,...,...
70,Online15,FirstFit,382.725,8.483468
71,Online15,BestFit,387.175,8.960795
72,Online15,WorstFit,343.830,8.935340
73,Online15,Greedy,412.870,6.656172


In [ ]:
mean_output = df_sim_comb.pivot(columns='algorithm', values=['seats', 'std']).loc[grid_order]
# mean_output.to_latex("results/table_output.tex")

The mean performance of each algorithm per grid:

In [16]:
bar_mean = px.bar(df_sim_comb, x='grid', color='algorithm', y='seats', barmode='group', error_y='std')
bar_mean.show()

### Best performance of each algorithm per grid:

In [17]:
df_sim_max = df_sim.groupby(['grid']).max().reset_index().melt(id_vars='grid', value_name='seats', var_name='algorithm').set_index('grid').loc[grid_order].reset_index()
px.bar(df_sim_max, x='grid', color='algorithm', y='seats', barmode='group').show()

### Ranking of algorithm in respect to each other per grid

In [18]:
# For each grid - group_sequence combination, rank the algorithms in respect to each other
df_sim.index.names = ['group_sequence', 'grid']
df_ranking = df_sim.reset_index().melt(id_vars=['group_sequence','grid'], value_name='seats', var_name='algorithm').sort_values(['grid', 'group_sequence'])
rank = df_ranking.groupby(['grid', 'group_sequence'])
rank = rank['seats'].rank(method='min', ascending=False)
df_ranking['rank'] = rank.astype('int16')
df_ranking # .head(n=50)

,group_sequence,grid,algorithm,seats,rank
0,1,Online1,FirstFit,4,4
3000,1,Online1,BestFit,5,3
6000,1,Online1,WorstFit,3,5
9000,1,Online1,Greedy,7,1
12000,1,Online1,Hybrid,7,1
...,...,...,...,...,...
2999,200,Online9,FirstFit,67,1
5999,200,Online9,BestFit,64,3
8999,200,Online9,WorstFit,62,4
11999,200,Online9,Greedy,66,2


Mean ranking of each algorithm per grid over all the simulations

In [20]:
# df_ranking_mean = df_ranking.groupby(['grid', 'algorithm']).mean().rename(columns={'rank': 'mean_rank'})
# rank = df_ranking_mean.groupby(['grid'])
# rank = rank['mean_rank'].rank(method='min', ascending=True).to_frame()
# # df_ranking_result = df_ranking_mean[rank]
# # df_ranking_result
# # df_ranking_mean
# df_ranking_result = rank.reset_index('algorithm').set_index('mean_rank', append=True).reset_index('grid').pivot(columns='grid') 
# df_ranking_result.columns = df_ranking_result.columns.droplevel(None)
# # print(grid_order)
# # print(df_ranking_result.columns)
# df_ranking_result = df_ranking_result[grid_order] 
# df_ranking_result

In [ ]:
# import plotly.graph_objects as go

# col_values = [df_ranking_result[col] for col in grid_order]
# table = go.Figure(
#     data=[
#         go.Table(
#             columnwidth = [500 for n in range(12)],
#             header=dict(values=list(df_ranking_result.columns)),
#             cells=dict(values=col_values),
#         )
#     ]
# )
# table.update_layout(autosize=False, width=2000)
# table.show()
# table.write_image('table_online.pdf')

In [ ]:
# px.scatter(df_ranking_mean.assign(**{'base': 5}), 
#     # base='base', 
#     x=df_ranking_mean.index.get_level_values(0), color=df_ranking_mean.index.get_level_values(1), y='rank').update_yaxes(range=[5, 1])

In [ ]:
# grid_rank = df_ranking.groupby(['grid', 'algorithm'])
# grid_rank = grid_rank['rank'].rank(method='min', ascending=False)
# grid_rank
# # ranking_per_grid = df_ranking['rank'] = grid_rank
# # ranking_per_grid

For each grid: the rank frequency of the algorithm:

In [ ]:
# # Rank per 
# def get_cum_ranking_per_rank(x):
#     result = x['rank'].value_counts().to_frame('count')
#     return result
# df_ranking_result = df_ranking.groupby(['grid', 'algorithm']).apply(get_cum_ranking_per_rank)
# df_ranking_result.index.names = ['grid', 'algorithm', 'rank']
# df_ranking_result = df_ranking_result.reset_index(level='rank').pivot(columns=['rank']).fillna(0).melt(ignore_index=False, var_name='rank', col_level=1)
# # print(df_ranking_result.info())
# # print(df_ranking_result.index.get_level_values(1))
# df_ranking_result
# # px.bar(df_ranking_result, x=df_ranking_result.index.get_level_values(0), color=df_ranking_result.index.get_level_values(1), y='rank', barmode='group').show()